# Colorize point cloud

In [1]:
import numpy as np
import cv2

In [2]:
basedir = '../SemanticKITTI/dataset/sequences/00'
pc_file = f'{basedir}/velodyne/000000.bin'
im_file = f'{basedir}/image_3/000000.png'
calib_file = f'{basedir}/calib.txt'


In [11]:
from colorize.colorize import doColorize

# Colorize with image_2
points2, _, _ = doColorize(pc_file, f'{basedir}/image_2/000000.png', calib_file, cam=0)
# Colorize with image_3
points3, _, _ = doColorize(pc_file, f'{basedir}/image_3/000000.png', calib_file, cam=1)

# get the valid array, make sure it the same size as points2
pt_valid2 = points2[:, 4] == 1
pt_valid3 = points3[:, 4] == 1
# Or combine the two valid arrays
combined_valid = pt_valid2 | pt_valid3
#  calculate the percentage of valid points
percent_valid = 100.0 * np.sum(combined_valid) / points2.shape[0]
print(f"Percentage of valid points: {percent_valid:.2f}%")
#  calculate the sum of valid points
sum_valid = np.sum(combined_valid)
print(f"Sum of valid points: {sum_valid}")
# percent_valid = 100.0 * np.sum(combined_valid) / points.shape[0]
# print(f"Percentage of valid points: {percent_valid:.2f}%")

Number of valid points/ Number of points: 19369/124668 (15.54%)
Number of valid points/ Number of points: 19481/124668 (15.63%)
Percentage of valid points: 15.95%
Sum of valid points: 19887


In [ ]:
from projection.spherical_proj import RangeProjection
# projector = RangeProjection(fov_up = 3.0, fov_down = -25.0, proj_w = 2048, proj_h = 64)
# Project the point cloud
# proj_pointcloud, proj_range, proj_idx, proj_mask = projector.doProjection(points)

from projection.scan_unfold import ScanProjection
scan_unfold = ScanProjection(proj_w = 2048, proj_h = 64)
proj_pointcloud, proj_range, proj_idx, proj_mask = scan_unfold.doProjection(points, wrap_around = 1024)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.imshow(proj_range, cmap='jet')
plt.title('Projected Range Image')
plt.colorbar(label='Range')
plt.axis('off')
plt.show()


In [ ]:
# plot the projected range image with color. If the valid is 1, set the color to the color of the point, otherwise set it to black

fig, axs = plt.subplots(3, 1, figsize=(18, 12))
# 1. Top: proj_range2 as heatmap (no colorbar)
axs[0].imshow(proj_range, cmap='jet')
axs[0].set_title('Projected Range Image')
axs[0].axis('off')

# 2. Middle: show the original image
img = cv2.imread(im_file)
# flip the image horizontally
img = cv2.flip(img, 1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
axs[1].imshow(img)
axs[1].set_title('Camera image (full resolution)')
axs[1].axis('off')

# 3. Bottom: heatmap with colored points overlaid
valid_mask = proj_pointcloud[..., 4] == 1
axs[2].imshow(proj_range, cmap='jet')
ys, xs = np.where(valid_mask)
colors_d = proj_pointcloud[ys, xs, 5:8]
axs[2].scatter(xs, ys, c=colors_d, s=2, marker='o')
axs[2].set_title('Range image with colorized point cloud Overlay')
axs[2].axis('off')

plt.tight_layout()
plt.show()
